In [13]:
import numpy as np
import random
from copy import deepcopy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import math,time,sys
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from functools import partial
import seaborn as sns 
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
# from sklearn.naive_bayes import 
# from sklearn.ensemble import RanfomForestClassifier
from sklearn.ensemble import RandomForestClassifier

MaxIter = 1
pop_size = 4
omega = 0.99
bp = 0.5
bsize = 1
wsize = 3

def initialise(partCount, dim, trainX, testX, trainy, testy):    
    population=np.zeros((partCount,dim))
    minn = 1
    maxx = math.floor(0.5*dim)
    
    if maxx<minn:
        maxx = minn + 1
        #not(c[i].all())
    
    for i in range(partCount):
        random.seed(i**3 + 10 + time.time() ) 
        no = random.randint(minn,maxx)
        if no == 0:
            no = 1
        random.seed(time.time()+ 100)
        pos = random.sample(range(0,dim-1),no)
        for j in pos:
            population[i][j]=1
            
    return population

def fitness(agent, trainX, testX, trainy, testy):
    # print(agent)
    cols=np.flatnonzero(agent)
    # print(cols)
    val=1
    if np.shape(cols)[0]==0:
        return val    
    clf=RandomForestClassifier(n_estimators=20)
    #clf=MLPClassifier(alpha=0.001, hidden_layer_sizes=(1000,500,100),max_iter=2000,random_state=4)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=1-clf.score(test_data,testy)

    #in case of multi objective  []
    set_cnt=sum(agent)
    set_cnt=set_cnt/np.shape(agent)[0]
    val=omega*val+(1-omega)*set_cnt
    return val

def allfit(population, trainX, testX, trainy, testy):
    x=np.shape(population)[0]
    acc=np.zeros(x)
    for i in range(x):
        acc[i]=fitness(population[i],trainX, testX, trainy, testy)     
        #print(acc[i])
    return acc

def test_accuracy(agent, trainX, testX, trainy, testy):
    cols=np.flatnonzero(agent)
    val=1
    if np.shape(cols)[0]==0:
        return val    
    # clf = RandomForestClassifier(n_estimators=300)
    #clf=MLPClassifier(alpha=0.001, hidden_layer_sizes=(1000,500,100),max_iter=2000,random_state=4)
    clf=KNeighborsClassifier(n_neighbors=5)
    # clf=MLPClassifier( alpha=0.01, max_iterno=1000) #hidden_layer_sizes=(1000,500,100)
    #cross=4
    #test_size=(1/cross)
    #X_train, X_test, y_train, y_test = train_test_split(trainX, trainy,  stratify=trainy,test_size=test_size)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=clf.score(test_data,testy)
    return val

def onecnt(agent):
    return sum(agent)

def sigmoid(gamma):
    if gamma < 0:
        return 1 - 1/(1 + math.exp(gamma))
    else:
        return 1/(1 + math.exp(-gamma))
    
def NMRA(dataset):
    df = pd.read_csv(dataset)
    a, b = np.shape(df)
    data = df.values[:,0:b-1]
    label = df.values[:,b-1]
    dimension = data.shape[1]
    
    cross = 5
    test_size = (1/cross)
    trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size,random_state=(7+17*int(time.time()%1000)))

    clf=RandomForestClassifier(n_estimators=20)
    clf.fit(trainX,trainy)

    val=clf.score(testX,testy)
    whole_accuracy = val
    print("Total Acc: ",val)
    
    pop = initialise(pop_size, dimension, trainX, testX, trainy, testy)
    fit = allfit(pop, trainX, testX, trainy, testy)
    
    ind = np.argsort(fit)
    index_b, index_w = [], []

    for i in range(bsize):
        index_b.append(ind[i])
        
    for i in range(wsize):
        index_w.append(ind[i])
    
    bestpop = pop[ind[0]]
    bestfit = fit[ind[0]]
    
    for i in range(MaxIter):
        for j in range(len(index_w)):
            l = []
            for k in range(dimension):
                random.seed(time.time()+k*2)
                l.append(random.random())
                
            pos = random.sample(range(0,len(index_w)-1),2)
            s = np.add(pop[index_w[j]], np.multiply(l, np.subtract(pop[index_w[pos[0]]],pop[index_w[pos[1]]])))
            
            if(fitness(s,trainX, testX, trainy, testy) < fit[index_w[j]]):
                fit[index_w[j]] = fitness(s,trainX, testX, trainy, testy)
                pop[index_w[j]] = s.copy()
                
                
        for j in range(len(index_b)):
            random.seed(time.time())
            if(random.random() < bp): 
                l, nl = [], []
                for k in range(dimension):
                    random.seed(time.time()+k*2)
                    l.append(random.random())

                for k in l:
                    nl.append(1-k)

                s = np.add(np.multiply(nl, pop[index_b[j]]), np.multiply(l, (np.subtract(bestpop, pop[index_b[j]]))))

                if(fitness(s,trainX, testX, trainy, testy) < fit[index_b[j]]):
                    fit[index_b[j]] = fitness(s,trainX, testX, trainy, testy)
                    pop[index_b[j]] = s.copy()  
                    
        for j in range(pop_size):
            for k in range(dimension):
                random.seed(time.time())
                if (sigmoid(pop[j][k]) > 0.5):
                    pop[j][k] = 1
                    
                else:
                    pop[j][k] = 0
                
                
        fit = allfit(pop, trainX, testX, trainy, testy)
        ind = np.argsort(fit)
        index_b, index_w = [], []

        for i in range(bsize):
            index_b.append(ind[i])

        for i in range(wsize):
            index_w.append(ind[i])

        bestpop = pop[ind[0]]
        bestfit = fit[ind[0]]
    
    fit = allfit(pop, trainX, testX, trainy, testy)
    ind = np.argsort(fit)
    bestpop = pop[ind[0]]
    bestfit = fit[ind[0]]
    
    testAcc = test_accuracy(bestpop, trainX, testX, trainy, testy)
    featCnt = onecnt(bestpop)
    #print("best agent: ", bestpop)
    print("Test Accuracy: ", testAcc)
    print("#Features: ", featCnt)
            
    return testAcc, featCnt, bestpop

In [14]:
%%time
Ta,fc,bp = NMRA('VoxForge512.csv')

Total Acc:  0.9777777777777777
Test Accuracy:  0.9
#Features:  36.0
CPU times: user 1.16 s, sys: 19.1 ms, total: 1.18 s
Wall time: 1.18 s


In [15]:
List = []

for i in range(0,len(bp)):
    if bp[i] == 1:
        List.append(i)
        
df_train = pd.read_csv("VoxForge512.csv")
y = (df_train['label'])
df_train1 = df_train[df_train.columns[List]]
l1 = []
for i in range(0,len(List)):
    l1.append(i)
    
df_train1.columns = l1

df_train1['label'] = y
df_train1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,label
0,0.004380,0.002665,0.000497,0.000497,0.000682,0.000609,0.001508,0.002521,0.001729,0.001222,...,0.503850,0.436740,11.858993,8.202133,0.974210,2.181164,15.391540,11.462402,0.016049,English
1,0.001584,0.002996,0.000250,0.000208,0.000248,0.000122,0.000278,0.001349,0.000641,0.000549,...,0.548493,0.212006,2.378749,5.564082,3.520760,6.403028,7.948945,11.421647,0.014899,English
2,0.001206,0.001195,0.000330,0.000584,0.000674,0.000366,0.000559,0.001241,0.000537,0.000703,...,0.452232,0.603910,1.479533,1.281132,2.430287,5.495424,8.274411,11.088083,0.014856,English
3,0.008713,0.008530,0.001264,0.001742,0.000852,0.001149,0.001520,0.005249,0.002413,0.002591,...,4.925777,0.588674,24.452608,27.830118,0.291790,0.230298,0.284989,9.626831,0.011871,English
4,0.004527,0.003138,0.000424,0.000823,0.000760,0.000430,0.001003,0.001894,0.001231,0.001088,...,0.774701,0.889606,15.672068,3.751125,0.687353,1.843793,5.087078,9.285885,0.011261,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.000559,0.000726,0.000355,0.000523,0.001019,0.000396,0.000470,0.000399,0.000657,0.000297,...,0.002010,0.003282,0.014107,0.009643,0.000996,0.003580,0.027846,29.929813,0.032924,Spanish
896,0.000161,0.000092,0.000126,0.000170,0.000105,0.000142,0.000113,0.000094,0.000065,0.000078,...,0.000989,0.001065,0.005478,0.008104,0.001649,0.005327,0.012645,9.986907,0.005506,Spanish
897,0.000040,0.000042,0.000036,0.000030,0.000055,0.000043,0.000037,0.000034,0.000052,0.000053,...,0.000620,0.000888,0.005123,0.008245,0.002862,0.003984,0.002771,9.939436,0.006557,Spanish
898,0.000005,0.000008,0.000010,0.000010,0.000012,0.000017,0.000010,0.000006,0.000011,0.000007,...,0.000186,0.000297,0.002047,0.002857,0.001005,0.003023,0.003097,9.965343,0.004320,Spanish


In [10]:
df_train1.to_csv('VF_BNMRA.csv')